# Import libraries
- ```pandas``` was imported to read ```.xlsx``` and convert to ```np.array```

In [109]:
import pandas as pd
import numpy as np
import time

# Define function $W_{F}$
### $W_{F}=\frac{WFF \times UCF \times LHV}{Ex_{F}^{0} \times \eta_{ex} \times Lt}$

In [110]:
# Eq 1
def WF(data):
    np.random.seed(0)

    wff = data[:, -1]

    # Assumptions in Step 3
    ucf = np.full_like(data[:, 0], 1.0)  # Unit conversion factors to 1
    lhv = np.full_like(data[:, 0], np.nan)
    exf = np.full_like(data[:, 0], np.nan)
    lt = np.full_like(data[:, 0], np.nan)
    nex = np.full_like(data[:, 0], np.nan)

    ucf[np.where(data[:, 1] == "Natural gas")] = 1
    ucf[np.where(data[:, 1] == "Coal")] = 1

    lhv[np.where(data[:, 1] == "Natural gas")] = 52.2
    lhv[np.where(data[:, 1] == "Coal")] = 30.2

    exf[np.where(data[:, 1] == "Natural gas")] = 0.052
    exf[np.where(data[:, 1] == "Coal")] = 0.034

    lt[
        np.where(
            (data[:, 1] == "Coal")
            | (data[:, 1] == "Natural gas")
            | (data[:, 1] == "Nuclear")
            | (data[:, 1] == "Geothermal")
        )
    ] = 30
    lt[
        np.where((data[:, 1] == "PV") | (data[:, 1] == "Wind") | (data[:, 1] == "CSP"))
    ] = 15

    # Sampled from uniform dist. with range given in Table 3
    nex[np.where(data[:, 1] == "Coal")] = np.random.uniform(
        0.218, 0.53, size=np.where(data[:, 1] == "Coal")[0].shape
    )
    nex[np.where(data[:, 1] == "Natural gas")] = np.random.uniform(
        0.17, 0.7, size=np.where(data[:, 1] == "Natural gas")[0].shape
    )
    nex[np.where(data[:, 1] == "Nuclear")] = np.random.uniform(
        0.2899, 0.461, size=np.where(data[:, 1] == "Nuclear")[0].shape
    )
    nex[np.where(data[:, 1] == "Geothermal")] = np.random.uniform(
        0.25, 0.83, size=np.where(data[:, 1] == "Geothermal")[0].shape
    )
    nex[np.where(data[:, 1] == "CSP")] = np.random.uniform(
        0.06, 0.597, size=np.where(data[:, 1] == "CSP")[0].shape
    )
    nex[np.where(data[:, 1] == "Wind")] = np.random.uniform(
        0.01, 0.92, size=np.where(data[:, 1] == "Wind")[0].shape
    )
    nex[np.where(data[:, 1] == "PV")] = np.random.uniform(
        0.0251, 0.15, size=np.where(data[:, 1] == "PV")[0].shape
    )

    """
    wff: given in excel
    ucf, lhv, exf, lt: given in Step 2
    nex: sampled from Table 3
    """
    Wf = (wff * ucf * lhv) / (exf * nex * lt)

    # Assumptions in Step 2
    Wf[np.where((data[:, 2] == "Operating") | (data[:, 2] == "Non-operating"))] = 0
    Wf[np.where(data[:, 1] == "Nuclear")] = 0
    # Wf = Wf[np.where((data[:, 1] == "Natural gas") | (data[:, 1] == "Coal"))]
    # data = data[np.where((data[:, 1] == "Natural gas") | (data[:, 1] == "Coal"))]

    return np.nan_to_num(Wf.astype(np.float64))

# Define function $W_{P}$
### $W_{P}=\frac{WFP \times UCF}{Av \times CF \times \eta_{ex} \times Lt}$

In [111]:
# Eq 2
def WP(data):
    np.random.seed(0)

    wfp = data[:, -1]

    # Assumptions in Step 3
    ucf = np.full_like(data[:, 0], 1.0)  # Unit conversion factors to 1
    av_cf = np.full_like(data[:, 0], 0.0)
    lt = np.full_like(data[:, 0], 0.0)
    nex = np.full_like(data[:, 0], 0.0)

    ucf[np.where(data[:, 1] == "Natural gas")] = 1
    ucf[np.where(data[:, 1] == "Coal")] = 1

    lt[
        np.where(
            (data[:, 1] == "Coal")
            | (data[:, 1] == "Natural gas")
            | (data[:, 1] == "Nuclear")
            | (data[:, 1] == "Geothermal")
        )
    ] = 30
    lt[
        np.where((data[:, 1] == "PV") | (data[:, 1] == "Wind") | (data[:, 1] == "CSP"))
    ] = 15

    # Sampled from uniform distribution in range given in Table 3
    av_cf[np.where(data[:, 1] == "Coal")] = np.random.uniform(
        0.6046, 1, size=np.where(data[:, 1] == "Coal")[0].shape
    )
    av_cf[np.where(data[:, 1] == "Natural gas")] = np.random.uniform(
        0.3476, 1, size=np.where(data[:, 1] == "Natural gas")[0].shape
    )
    av_cf[np.where(data[:, 1] == "Nuclear")] = np.random.uniform(
        0.905, 1, size=np.where(data[:, 1] == "Nuclear")[0].shape
    )
    av_cf[np.where(data[:, 1] == "Geothermal")] = np.random.uniform(
        0.7838, 1, size=np.where(data[:, 1] == "Geothermal")[0].shape
    )
    av_cf[np.where(data[:, 1] == "CSP")] = np.random.uniform(
        0.0448, 1, size=np.where(data[:, 1] == "CSP")[0].shape
    )
    av_cf[np.where(data[:, 1] == "Wind")] = np.random.uniform(
        0.122, 1, size=np.where(data[:, 1] == "Wind")[0].shape
    )
    av_cf[np.where(data[:, 1] == "PV")] = np.random.uniform(
        0.0448, 1, size=np.where(data[:, 1] == "PV")[0].shape
    )

    nex[np.where(data[:, 1] == "Coal")] = np.random.uniform(
        0.218, 0.53, size=np.where(data[:, 1] == "Coal")[0].shape
    )
    nex[np.where(data[:, 1] == "Natural gas")] = np.random.uniform(
        0.17, 0.7, size=np.where(data[:, 1] == "Natural gas")[0].shape
    )
    nex[np.where(data[:, 1] == "Nuclear")] = np.random.uniform(
        0.2899, 0.461, size=np.where(data[:, 1] == "Nuclear")[0].shape
    )
    nex[np.where(data[:, 1] == "Geothermal")] = np.random.uniform(
        0.25, 0.83, size=np.where(data[:, 1] == "Geothermal")[0].shape
    )
    nex[np.where(data[:, 1] == "CSP")] = np.random.uniform(
        0.06, 0.597, size=np.where(data[:, 1] == "CSP")[0].shape
    )
    nex[np.where(data[:, 1] == "Wind")] = np.random.uniform(
        0.01, 0.92, size=np.where(data[:, 1] == "Wind")[0].shape
    )
    nex[np.where(data[:, 1] == "PV")] = np.random.uniform(
        0.0251, 0.15, size=np.where(data[:, 1] == "PV")[0].shape
    )

    Wp = (wfp * ucf) / (av_cf * nex * lt)

    return np.nan_to_num(Wp.astype(np.float64))

# Define functions for randomly weighted summation
### $FU \times W_{F}$ and $RV \times W_{P}$

In [112]:
def RandomWeightedSum(wf, wp, data):
    wf = wf.reshape(-1, 1)
    wp = wp.reshape(-1, 1)
    # np.random.seed(0)

    # Random values according to Step 4
    rv = np.ones((data.shape[0], 10)).astype(np.float64)
    fu = np.ones((data.shape[0], 10)).astype(np.float64)

    rv[np.where(data[:, 1] == "CSP")] = np.random.uniform(
        0.8, 1.2, size=(np.where(data[:, 1] == "CSP")[0].shape[0], 10)
    )
    rv[np.where(data[:, 1] == "Wind")] = np.random.uniform(
        0.8, 1.2, size=(np.where(data[:, 1] == "Wind")[0].shape[0], 10)
    )
    rv[np.where(data[:, 1] == "PV")] = np.random.uniform(
        0.8, 1.2, size=(np.where(data[:, 1] == "PV")[0].shape[0], 10)
    )

    fu[np.where(data[:, 1] == "Coal")] = np.random.uniform(
        1, 1.029, size=(np.where(data[:, 1] == "Coal")[0].shape[0], 10)
    )
    fu[np.where(data[:, 1] == "Natural gas")] = np.random.uniform(
        1, 1.049, size=(np.where(data[:, 1] == "Natural gas")[0].shape[0], 10)
    )
    fu[np.where(data[:, 1] == "Nuclear")] = np.random.uniform(
        1, 1.039, size=(np.where(data[:, 1] == "Nuclear")[0].shape[0], 10)
    )

    Wf = (fu * wf).sum(axis=1)
    Wp = (rv * wp).sum(axis=1)

    return Wf, Wp

# Define main function

In [113]:
def Main(verbose: bool = True):
    # Used pandas only for reading xlsx file
    data = pd.read_excel("data/Power plot data.xlsx").to_numpy()
    
    technologies = ["Coal", "Natural gas", "Nuclear", "Geothermal", "PV", "Wind", "CSP"]

    wf = WF(data)
    wp = WP(data)
    wf, wp = RandomWeightedSum(wf, wp, data)

    # sum values
    wf_final = {"Consumption": [], "Withdrawal": []}
    wp_final = {"Consumption": [], "Withdrawal": []}
    for k in wf_final.keys():
        for c in technologies:
            wf_final[k].append(
                wf[np.where((data[:, 1] == c) & (data[:, 0] == k))].sum().round(2)
            )
            wp_final[k].append(
                wp[np.where((data[:, 1] == c) & (data[:, 0] == k))].sum().round(2)
            )

    # Print
    if verbose:
        index = ["WP", "WF"]
        for idx_v, value in enumerate([wp_final.items(), wf_final.items()]):
            for k, v in value:
                for idx_t, t in enumerate(technologies):
                    print(f"{index[idx_v]}_{k}_{t}: {v[idx_t]}")
            print()

# Run function ```Main()```

In [114]:
Main()

WP_Consumption_Coal: 243933.53
WP_Consumption_Natural gas: 192480.71
WP_Consumption_Nuclear: 35201.92
WP_Consumption_Geothermal: 2653.83
WP_Consumption_PV: 946243.23
WP_Consumption_Wind: 7013.56
WP_Consumption_CSP: 2353473.31
WP_Withdrawal_Coal: 5665179.15
WP_Withdrawal_Natural gas: 2472370.63
WP_Withdrawal_Nuclear: 1946655.8
WP_Withdrawal_Geothermal: 48683.28
WP_Withdrawal_PV: 23998467.01
WP_Withdrawal_Wind: 96394.95
WP_Withdrawal_CSP: 345373.51

WF_Consumption_Coal: 4199542.3
WF_Consumption_Natural gas: 4068001.96
WF_Consumption_Nuclear: 0.0
WF_Consumption_Geothermal: 0.0
WF_Consumption_PV: 0.0
WF_Consumption_Wind: 0.0
WF_Consumption_CSP: 0.0
WF_Withdrawal_Coal: 59759212.7
WF_Withdrawal_Natural gas: 35551417.32
WF_Withdrawal_Nuclear: 0.0
WF_Withdrawal_Geothermal: 0.0
WF_Withdrawal_PV: 0.0
WF_Withdrawal_Wind: 0.0
WF_Withdrawal_CSP: 0.0



# Measuring code run time
Measured on MacBook Pro with M1 Pro CPU running OSX Ventura 13.6

In [115]:
repeat = 100
elapsed = 0
for _ in range(repeat):
    t1 = time.perf_counter()
    Main(verbose=False)
    t2 = time.perf_counter()
    elapsed += t2 - t1
print(f"{repeat} repeated runs average: {elapsed/repeat:.4f}s")